In [1]:
## install environment
# !conda install --file ../env/byoml_custom/environment.yml
# !conda activate byoml_custom
# !pip install -r ../env/byoml_custom/requirements.txt

In this notebook we will demonstrate how to create your own simple custom models to upload to Waylay BYOML. We will create a function which doubles the inputs.
The dependencies in this environment are:
* waylay-beta
* dill

A best practice for serializing models is using `Python 3.7`, since the latest `dill` version is not yet up to date with newer versions of Python.

In [3]:
MODEL_NAME = 'test_custom_model'
PROFILE = 'render_notebook'

In [4]:
import numpy as np

## Custom model class

The following cell contains the custom model which has to contain a `predict` method. We would also like to map the predictions to our custom mapping function, all we have to do for this is create a method called `map_output`.

In [5]:
class Doubler:
    def __init__(self):
        self.keys = ['foo', 'bar', 'baz', 'something', 'else']
        
    def predict(self, data):
        return data * 2
    
    def map_output(self, request, predictions):
        mapped_predictions = [
            {
                self.keys[idx]: value
                for idx, value in enumerate(pred)
            }
            for pred in predictions.tolist()
        ]
        return {'predictions': mapped_predictions}

We test the class locally with an example input.

In [6]:
X = np.array([[1, 2, 3, 4, 5]])

In [7]:
doubler = Doubler()
preds = doubler.predict(X)
preds

array([[ 2,  4,  6,  8, 10]])

In [8]:
doubler.map_output({}, preds)

{'predictions': [{'foo': 2, 'bar': 4, 'baz': 6, 'something': 8, 'else': 10}]}

## Model upload

When the model is ready, we can upload it to the platform. When uploading the model we need to provide some basic information. Like:

* which framework: `sklearn`, `tensorflow`, `pytorch` or `xgboost` or `custom`
* a name for the model
* the model file (or directory for TensorFlow models)
* an optional description

In [9]:
from waylay import WaylayClient
waylay = WaylayClient.from_profile(PROFILE)

In [10]:
waylay.byoml.model.upload(
        model_name=MODEL_NAME,
        trained_model=doubler,
        framework='custom',
        description='Simple custom model example'
    )

{'createdOn': '2021-01-04T00:00:00+00:00',
 'description': 'Simple custom model example',
 'framework': 'custom',
 'framework_version': '1.0',
 'message': 'Model successfully uploaded',
 'metadata': {'description': 'Simple custom model example'},
 'name': 'test_custom_model',
 'updatedOn': '2021-01-04T00:00:00+00:00',
 'version': '1'}

We can retrieve information about the uploaded model, like:
* creator id
* description
* framework
* readiness

In [11]:
waylay.byoml.model.get(MODEL_NAME)

{'createdBy': 'users/6d686630-0771-4f92-8c63-c888146c33fc',
 'createdOn': '2022-09-06T10:51:37.045000+00:00',
 'description': 'Simple custom model example',
 'framework': 'custom',
 'framework_version': '1.0',
 'metadata': {'description': 'Simple custom model example'},
 'name': 'test_custom_model',
 'ready': False,
 'supported_endpoints': [],
 'updatedBy': 'users/6d686630-0771-4f92-8c63-c888146c33fc',
 'updatedOn': '2022-09-06T10:51:57.859000+00:00',
 'version': '1'}

## Model test

We can create a test prediction and validate the performance of the mapped results.

In [12]:
X_test = [[1, 2, 3, 4, 5], [10, 20, 30, 40, 50]]

In [13]:
waylay.byoml.model.predict(MODEL_NAME, X_test)

[{'foo': 2, 'bar': 4, 'baz': 6, 'something': 8, 'else': 10},
 {'foo': 20, 'bar': 40, 'baz': 60, 'something': 80, 'else': 100}]

In [14]:
waylay.byoml.model.remove(MODEL_NAME)

{'message': 'Model test_custom_model: all versions deleted',
 'name': 'test_custom_model',
 'versions': ['0.0.1']}